In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from scipy.stats import expon, reciprocal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
import deap
import skopt
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import save_model
from tensorflow.keras.optimizers import SGD

In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import joblib

def preprocess_data(file_path, is_train=True):
    df = pd.read_csv(file_path)
    
    # Convert binary categorical features to 0 and 1
    binary_features = ['CryoSleep', 'VIP']
    df[binary_features] = df[binary_features].astype(bool).astype(int)
    
    # Feature Engineering
    df['TotalSpending'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    
    # Conditionally set spending-related features to 0 for passengers in cryosleep
    spending_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    df.loc[df['CryoSleep'] == 1, spending_features] = 0
    
    # Create interaction features
    df['HomePlanet_TotalSpending'] = df['HomePlanet'].astype(str) + '_' + df['TotalSpending'].astype(str)
    df['Destination_TotalSpending'] = df['Destination'].astype(str) + '_' + df['TotalSpending'].astype(str)
    
    # Extract components from 'Cabin'
    if 'Cabin' in df.columns:
        df[['Cabin_Deck', 'Cabin_Number', 'Cabin_Side']] = df['Cabin'].str.split('/', expand=True)
        df['Cabin_Side'] = df['Cabin_Side'].map({'P': 1, 'S': 0})
        df['Cabin_Number'] = pd.to_numeric(df['Cabin_Number'], errors='coerce')
        df.drop('Cabin', axis=1, inplace=True)
    
    # One-hot encode multi-category features
    multi_cat_features = ['HomePlanet', 'Destination']
    if is_train:
        one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
        encoded_features = one_hot_encoder.fit_transform(df[multi_cat_features])
        encoded_feature_names = one_hot_encoder.get_feature_names_out(multi_cat_features)
        joblib.dump(one_hot_encoder, 'one_hot_encoder.pkl')
    else:
        one_hot_encoder = joblib.load('one_hot_encoder.pkl')
        encoded_features = one_hot_encoder.transform(df[multi_cat_features])
        encoded_feature_names = one_hot_encoder.get_feature_names_out(multi_cat_features)
    
    encoded_features_df = pd.DataFrame(encoded_features.toarray(), columns=encoded_feature_names)
    df = pd.concat([df, encoded_features_df], axis=1)
    df.drop(multi_cat_features, axis=1, inplace=True)
    
    # Imputation and Scaling
    numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin_Number', 'Cabin_Side', 'TotalSpending']
    if is_train:
        imputer = KNNImputer(n_neighbors=5)
        scaler = StandardScaler()
        df[numeric_features] = imputer.fit_transform(df[numeric_features])
        df[numeric_features] = scaler.fit_transform(df[numeric_features])
        joblib.dump(imputer, 'imputer.pkl')
        joblib.dump(scaler, 'scaler.pkl')
    else:
        imputer = joblib.load('imputer.pkl')
        scaler = joblib.load('scaler.pkl')
        df[numeric_features] = imputer.transform(df[numeric_features])
        df[numeric_features] = scaler.transform(df[numeric_features])
    
    if is_train:
        # Convert 'Transported' to integer (True=1, False=0) for modeling
        df['Transported'] = df['Transported'].astype(int)
        
        # Save the list of features used for training
        train_features = [col for col in df.columns if col not in ['PassengerId', 'Name', 'Transported', 'Cabin_Deck', 'HomePlanet_TotalSpending', 'Destination_TotalSpending']]
        joblib.dump(train_features, 'train_features.pkl')
    
    return df

In [60]:
train_df = preprocess_data('csv_files/train.csv', is_train=True)
test_df = preprocess_data('csv_files/test.csv', is_train=False)

# Function to create the model, now with an optimizer parameter
def create_model(input_shape, layers, activation, dropout_rate, learning_rate, optimizer_type='adam'):
    model = Sequential()
    model.add(Dense(layers[0], activation=activation, input_shape=(input_shape,)))
    model.add(Dropout(dropout_rate))
    
    for layer_size in layers[1:]:
        model.add(Dense(layer_size, activation=activation))
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))

    # Choose the optimizer
    if optimizer_type == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        optimizer = Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Define a schedule function
def lr_schedule(epoch, lr):
    if epoch > 0 and epoch % 10 == 0:
        return lr * 0.9
    return lr

def train_and_evaluate(model, X_train, y_train, X_val, y_val, batch_size, epochs, class_weight):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    learning_rate_scheduler = LearningRateScheduler(lr_schedule)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
              validation_data=(X_val, y_val), class_weight=class_weight, 
              callbacks=[early_stopping, learning_rate_scheduler])
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    return val_loss, val_acc

# Prepare features and target for the model
features = [col for col in train_df.columns if col not in ['PassengerId', 'Name', 'Transported', 'Cabin_Deck', 'HomePlanet_TotalSpending', 'Destination_TotalSpending']]
X = train_df[features]
y = train_df['Transported']

# Splitting the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Handling class imbalance
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_smote),
    y=y_train_smote)
class_weight_dict = dict(enumerate(class_weights))

# Optimization loop
target_accuracy = 0.85
max_iterations = 5
current_iteration = 0
best_val_acc = 0
best_params = {'layers': [128, 128, 64], 'activation': 'tanh', 'dropout_rate': 0.3, 'learning_rate': 0.001}
# Initialization outside the loop
best_model = None

while current_iteration < max_iterations:
    # Use best parameters from previous iterations
    layers = best_params['layers']
    activation = best_params['activation']
    dropout_rate = best_params['dropout_rate']
    learning_rate = best_params['learning_rate']
    
    # Here you can decide to use SGD or Adam based on some condition, e.g., alternate between them
    if current_iteration % 2 == 0:
        optimizer_type = 'adam'
    else:
        optimizer_type = 'sgd'
    
    # Create and train the model
    model = create_model(input_shape=X_train_smote.shape[1], 
                         layers=layers, 
                         activation=activation, 
                         dropout_rate=dropout_rate, 
                         learning_rate=learning_rate,
                         optimizer_type=optimizer_type)

    val_loss, val_acc = train_and_evaluate(model, X_train_smote, y_train_smote, X_val, y_val, 
                                           batch_size=16, epochs=100, class_weight=class_weight_dict)

    print(f'Iteration {current_iteration} using {optimizer_type.upper()}: Validation Accuracy = {val_acc}')

    # Update best parameters if current model is better
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_params = {'layers': layers, 'activation': activation, 'dropout_rate': dropout_rate, 'learning_rate': learning_rate, 'optimizer_type': optimizer_type}
        best_model = model  
        print("New best model found and saved.")

    # Adjust model complexity based on performance
    if val_acc < best_val_acc - 0.05:  # significant drop in performance
        layers = [max(l // 2, 32) for l in layers]  # reduce complexity
    elif val_acc > best_val_acc:
        layers = [min(l * 2, 512) for l in layers]  # increase complexity if showing improvement

    # Try different activation functions
    if current_iteration % 2 == 0:
        activation = 'relu'
    else:
        activation = 'tanh'

    # Adjust dropout rate based on overfitting/underfitting
    if val_acc < best_val_acc - 0.03:  # underfitting
        dropout_rate = max(dropout_rate - 0.1, 0)
    elif val_acc > best_val_acc + 0.03:  # overfitting
        dropout_rate = min(dropout_rate + 0.1, 0.5)

    # Adjust learning rate based on performance
    if val_acc < best_val_acc - 0.02:
        learning_rate *= 0.5  # reduce learning rate if performance drops
    elif val_acc > best_val_acc + 0.02:
        learning_rate *= 1.2  # increase learning rate if performance improves

    if best_val_acc >= target_accuracy:
        print("Target accuracy reached.")
        break

    current_iteration += 1

# Final evaluation
if best_val_acc < target_accuracy:
    print("Target accuracy not reached after maximum iterations.")
    print(f"Final Model Validation Accuracy: {best_val_acc}")
else:
    print(f"Final Model Validation Accuracy: {best_val_acc}")

model_enhanced = best_model
# Combine the original training and validation sets for final training
X_full, y_full = smote.fit_resample(X, y)
model_enhanced.fit(X_full, y_full, batch_size=16, epochs=100, class_weight=class_weight_dict)

Epoch 1/100


/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 601us/step - accuracy: 0.7438 - loss: 0.5113 - val_accuracy: 0.7671 - val_loss: 0.4722 - learning_rate: 0.0010
Epoch 2/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - accuracy: 0.7802 - loss: 0.4626 - val_accuracy: 0.7729 - val_loss: 0.4618 - learning_rate: 0.0010
Epoch 3/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.7850 - loss: 0.4635 - val_accuracy: 0.7752 - val_loss: 0.4745 - learning_rate: 0.0010
Epoch 4/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - accuracy: 0.7852 - loss: 0.4461 - val_accuracy: 0.7763 - val_loss: 0.4724 - learning_rate: 0.0010
Epoch 5/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - accuracy: 0.7903 - loss: 0.4401 - val_accuracy: 0.7803 - val_loss: 0.4590 - learning_rate: 0.0010
Epoch 6/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.7854 - loss: 0.4536 - val_accuracy: 0.7769 - val_loss: 0.4628 - learning_rate: 0.0010
Epoch 7/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step - accuracy: 0.7976 - loss: 

In [66]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import joblib

# Load the best model
model_enhanced = load_model('model_enhanced.keras')

# Preprocess the test data using the same steps as the training data
test_df = preprocess_data('csv_files/test.csv', is_train=False)

# Load the features used during training
train_features = joblib.load('train_features.pkl')

# Select the features used by the model, ensuring they are the same as those used in training
features = [col for col in train_features if col in test_df.columns]

# Ensure that X_test is a DataFrame with the correct numerical types
X_test = test_df[features].copy()

# Convert X_test to a NumPy array, which is the expected format for TensorFlow models
X_test_np = X_test.values

# Predict using the enhanced model
y_pred_test_proba = model_enhanced.predict(X_test_np)
y_pred_test = (y_pred_test_proba > 0.5).astype(int).flatten()

# Prepare the submission dataframe
submission_df = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_pred_test
})

# Convert predictions back to boolean (True/False) if necessary
submission_df['Transported'] = submission_df['Transported'].astype(bool)

# Save the submission file
submission_df.to_csv('enhanced_model_submission.csv', index=False)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_1127" is incompatible with the layer: expected axis -1 of input shape to have value 16, but received input with shape (32, 19)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 19), dtype=float32)
  • training=False
  • mask=None

In [91]:
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

# Define the space of hyperparameters to search
space = [
    Integer(32, 512, name='layer_size'),
    Real(0.01, 0.5, name='dropout_rate'),
    Real(1e-4, 1e-2, prior='log-uniform', name='learning_rate')
]

# Function to optimize (we aim to minimize 1 - accuracy)
@use_named_args(space)
def objective(**params):
    model = create_model(
        input_shape=X_train_smote.shape[1], 
        layers=[params['layer_size']] * 2,  # Example: using the same size for two layers
        activation='relu', 
        dropout_rate=params['dropout_rate'], 
        learning_rate=params['learning_rate']
    )
    val_loss, val_acc = train_and_evaluate(model, X_train_smote, y_train_smote, X_val, y_val, 
                                           batch_size=32, epochs=50, class_weight=class_weight_dict)
    return 1 - val_acc  # as we need to minimize

# Run Bayesian optimization
result = gp_minimize(objective, space, n_calls=20, random_state=0)

print(f"Best parameters: {result.x}")
print(f"Best validation accuracy: {1 - result.fun}")

Epoch 1/50


/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7383 - loss: 0.5151 - val_accuracy: 0.7792 - val_loss: 0.4555
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7841 - loss: 0.4496 - val_accuracy: 0.7815 - val_loss: 0.4501
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - accuracy: 0.7928 - loss: 0.4447 - val_accuracy: 0.7809 - val_loss: 0.4328
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - accuracy: 0.7981 - loss: 0.4403 - val_accuracy: 0.7826 - val_loss: 0.4323
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - accuracy: 0.7954 - loss: 0.4219 - val_accuracy: 0.7849 - val_loss: 0.4416
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - accuracy: 0.8035 - loss: 0.4206 - val_accuracy: 0.7821 - val_loss: 0.4439
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - accuracy: 0.7999 - loss: 0.4246 - val_accuracy: 0.7821 - val_loss: 0.4326
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - accuracy: 0.7995 - loss: 0.4182 - val_accuracy: 0

/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 657us/step - accuracy: 0.6116 - loss: 0.6551 - val_accuracy: 0.7556 - val_loss: 0.5238
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - accuracy: 0.7415 - loss: 0.5324 - val_accuracy: 0.7665 - val_loss: 0.4774
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.7674 - loss: 0.4920 - val_accuracy: 0.7729 - val_loss: 0.4641
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.7806 - loss: 0.4744 - val_accuracy: 0.7740 - val_loss: 0.4592
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.7732 - loss: 0.4679 - val_accuracy: 0.7786 - val_loss: 0.4569
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step - accuracy: 0.7893 - loss: 0.4508 - val_accuracy: 0.7803 - val_loss: 0.4541
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.7816 - loss: 0.4648 - val_accuracy: 0.7798 - val_loss: 0.4494
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.7940 - loss: 0.4409 - val_accurac

...

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 687us/step - accuracy: 0.7269 - loss: 0.5447 - val_accuracy: 0.7803 - val_loss: 0.4599
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 443us/step - accuracy: 0.7851 - loss: 0.4460 - val_accuracy: 0.7792 - val_loss: 0.4534
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step - accuracy: 0.7922 - loss: 0.4425 - val_accuracy: 0.7838 - val_loss: 0.4404
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - accuracy: 0.7933 - loss: 0.4341 - val_accuracy: 0.7895 - val_loss: 0.4361
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.7990 - loss: 0.4161 - val_accuracy: 0.7849 - val_loss: 0.4351
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step - accuracy: 0.7960 - loss: 0.4295 - val_accuracy: 0.7815 - val_loss: 0.4321
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step - accuracy: 0.8003 - loss: 0.4252 - val_accuracy: 0.7924 - val_loss: 0.4259
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 529us/step - accuracy: 0.8002 - loss: 0.4173 - val_accurac

/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7603 - loss: 0.5087 - val_accuracy: 0.7602 - val_loss: 0.4882
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.7896 - loss: 0.4519 - val_accuracy: 0.7769 - val_loss: 0.4447
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.7976 - loss: 0.4271 - val_accuracy: 0.7878 - val_loss: 0.4496
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.7935 - loss: 0.4374 - val_accuracy: 0.7867 - val_loss: 0.4490
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.7993 - loss: 0.4313 - val_accuracy: 0.7815 - val_loss: 0.4459
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 954us/step - accuracy: 0.7948 - loss: 0.4343 - val_accuracy: 0.7867 - val_loss: 0.4233
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - accuracy: 0.7950 - loss: 0.4211 - val_accuracy: 0.7844 - val_loss: 0.4358
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - accuracy: 0.7971 - loss: 0.4218 - val_accuracy:

/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7465 - loss: 0.4995 - val_accuracy: 0.7608 - val_loss: 0.4853
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - accuracy: 0.7937 - loss: 0.4231 - val_accuracy: 0.7729 - val_loss: 0.4409
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - accuracy: 0.7936 - loss: 0.4212 - val_accuracy: 0.7792 - val_loss: 0.4421
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - accuracy: 0.7951 - loss: 0.4211 - val_accuracy: 0.7826 - val_loss: 0.4365
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8046 - loss: 0.4118 - val_accuracy: 0.7815 - val_loss: 0.4410
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8121 - loss: 0.4029 - val_accuracy: 0.7924 - val_loss: 0.4283
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8059 - loss: 0.4002 - val_accuracy: 0.7872 - val_loss: 0.4276
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8093 - loss: 0.3910 - val_accuracy: 0.7901 